<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/voyant_tools/post_tokens_with_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. This script reads a file with VK group post texts (created using the vk_group_all_posts.ipynb script).

2. Tokenizes (breaks into words), saves periods in sentences.

3. Combines posts from several VK groups into one CSV file. Adds a group field with the name of the VK group. Group posts are sorted by the date field.

The resulting CSV file is ready to be uploaded to Voyant Tools. The text (the tokens field) is enclosed in quotation marks specifically for this purpose.

---
In Russian

1. Этот скрипт читает файл с текстами постов группы ВК (созданный с помощью скрипта vk_group_all_posts.ipynb).

2. Токенизирует (разбивает на слова), сохраняет точки в предложениях.

3. Объединяет посты нескольких ВК-групп в один CSV-файл. Добавляет поле group с именем ВК-группы. Посты групп отсортированы по полю date.

Итоговый CSV-файл готов к загрузке в Voyant Tools. Специально для этого текст (поле tokens) заключён в кавычки.

In [37]:
domains = ['club221681617','concerto','club151359929','pravoslav_karelia']# smallest groups for tests
result_file = 'tokens_group.csv'

In [38]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
!pip install -U pymorphy3
import pymorphy3
import requests
import csv

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
alphabet_dash = alphabet | {'-'}# alphabet + dash

morph = pymorphy3.MorphAnalyzer(lang='ru')

In [39]:
def get_text_window(words, index, window_size=3):
    """Returns a context window of words around the given index."""
    start = max(0, index - window_size)
    end = min(len(words), index + window_size + 1)
    return ' '.join(words[start:end])

def contains_non_dash(s):
    """Check if a string consists not only dash characters."""
    return s.count('-') < len(s)

def process_text(text):
    sentences = sent_tokenize(text)  # Split into sentences
    processed_sentences = []

    for sentence in sentences:
        check_hash = False
        processed_parts = []
        words = word_tokenize(sentence)

        for i, w in enumerate(words):
          if len(w) == 1:
            continue
          if w == '#':
            check_hash = True
            continue
          if check_hash:
            check_hash = False
            continue

          # skip name and surname
          # w_tag = morph.parse(w.strip())[0].tag
          #if 'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
          #  context = get_text_window(words, i)
          #  print(f"Filtered name/surname: {w} | Context: {context}")  # Debug output for context
          #  continue

          if set(w.lower()).issubset(alphabet_dash) and contains_non_dash(w):
            res = morph.parse(w.lower())[0].normal_form
            if res and (res not in stop_words):
                  processed_parts.append(res)
          else:
            # has 4+ Cyrillic characters then will parse too (e.g. блж.Фаддея о.Алексия г.Петрозаводске)
            if sum(1 for char in w.lower() if char in alphabet) >= 4:
              if ('\\' not in w) and ('/' not in w): # skip words-hyperlinks
                #context = get_text_window(words, i)
                #print(f"Filtered not subset(alphabet): {w} | Context: {context}")
                res = morph.parse(w.lower())[0].normal_form
                if res not in stop_words:
                  processed_parts.append(res)

        if processed_parts:
            last_word = processed_parts[-1]
            if last_word[-1] not in ".!?":
                processed_parts.append(".")  # Add period at the end of sentence

        processed_sentences.append(" ".join(processed_parts))

    return " ".join(processed_sentences)

In [40]:
temp_df = []
for domain in domains:
    t = pd.read_csv(f'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/religion/{domain}.csv',
                    usecols=['text', 'date'])
    t['group'] = domain
    temp_df.append(t)

df = pd.concat(temp_df, ignore_index=True).sort_values('date')
df = df[df['text'].notna() & (df['text'].apply(lambda x: isinstance(x, str))) & (df['text'] != '')]

df['tokens'] = df['text'].apply(process_text)
df = df[df['tokens'].notna() & (df['tokens'].apply(lambda x: isinstance(x, str))) & (df['tokens'] != '') & (df['tokens'] != ' ')]
df_tokens = pd.concat([df['tokens'], df['date'],df['group']], axis=1, keys=['tokens', 'date','group'])

# Removing lines with empty 'tokens'
df_tokens = df_tokens[df_tokens['tokens'].str.strip().astype(bool)]

# Save CSV with quotes only for 'tokens' field, without quotes for 'date' and 'group'
with open(result_file, 'w', newline='', encoding='utf-8') as f:
#    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONE, escapechar='\\')
    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_NONE, quotechar=None)
    writer.writerow(['tokens', 'date', 'group'])
    for _, row in df_tokens.iterrows():
        writer.writerow([f'"{row["tokens"]}"', row['date'], row['group']])